## Libraries import

In [2]:
import os

# [Problem 1] Learning / Estimation

### clone code

In [2]:
os.chdir('/content/drive/MyDrive/Colab-Notebooks/')

In [3]:
! pwd

/content/drive/MyDrive/Colab-Notebooks


In [ ]:
! git clone https://github.com/zhixuhao/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.92 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (253/253), done.


In [3]:
os.chdir('/content/drive/MyDrive/Colab-Notebooks/unet')

In [4]:
from model import *
from data import *

### training phase

The dataset should have 2 type of images: real images and masks.

In [ ]:
# do data augmentation
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
# create unet model
model = unet()
#save parameters to .hdf5 file
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#train
model.fit_generator(myGene,steps_per_epoch=100,epochs=5,callbacks=[model_checkpoint])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
 40/100 [===========>..................] - ETA: 14:14 - loss: 0.5104 - accuracy: 0.7664

KeyboardInterrupt: ignored

Because the training time tooks a long time, I skipped. As I know, after finishing the training, the parameters is stored in ```unet_membrane.hdf5``` file. Then, in the testing phase, we load that file and predict the testing data.

### testing phase

In [ ]:
#load testset
testGene = testGenerator("data/membrane/test")
#create training model
model = unet()
#load parameters from .hdf5 file from previous training
model.load_weights("unet_membrane.hdf5")
#predict
results = model.predict_generator(testGene,30,verbose=1)
#save the predicted result
saveResult("data/membrane/test",results)

# [Problem 2] Code reading

<b>The architecture of the U-net</b>

Location: ```model.py```

Function: 
```python
def unet(pretrained_weights = None,input_size = (256,256,1))
```

<b>How up-conv is implemented by using Conv2D and UpSampling2D</b>

Location: ```model.py```

Function: 
```python
 up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
```

<b>Using Adam and binary_crossentropy for loss function, for binary classification</b>

Location: ```model.py```

Function: 

```python
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
```

<b>concatination on contradict path and expensive path</b>

Location: ```model.py```

Function: 

```python
merge7 = concatenate([conv3,up7], axis = 3)
```

<b>generate training data by loading images and masks</b>

Location: ```data.py```

```python
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
```